In [0]:
%sql
USE CATALOG kailyn_klaassen;
USE odcs;
CREATE OR REPLACE TABLE plant_operations_kpi_gold AS
-- 1) Battery KPIs per plant, per hour
WITH battery_hourly AS (
  SELECT
    plant_id,
    window_start,
    AVG(state_of_charge_pct)       AS avg_soc_pct,
    MIN(state_of_charge_pct)       AS min_soc_pct,
    MAX(state_of_charge_pct)       AS max_soc_pct,
    AVG(state_of_health_pct)       AS avg_soh_pct,
    SUM(energy_mwh)                AS batt_energy_mwh,
    AVG(temperature_c)             AS batt_avg_temp_c,
    SUM(charge_discharge_cycles)   AS batt_total_cycles
  FROM (
    SELECT
      plant_id,
      state_of_charge_pct,
      state_of_health_pct,
      energy_mwh,
      temperature_c,
      charge_discharge_cycles,
      window(timestamp, '1 hour') AS ts_win
    FROM battery_sensor_feed_silver
  )
  LATERAL VIEW explode(array(ts_win.start)) w AS window_start
  GROUP BY plant_id, window_start
),

-- 2) Solar KPIs per plant, per hour
solar_hourly AS (
  SELECT
    plant_id,
    window_start,
    SUM(energy_mwh)          AS solar_energy_mwh,
    AVG(power_mw)            AS solar_avg_power_mw,
    AVG(performance_ratio)   AS solar_avg_pr,
    AVG(inverter_efficiency_pct) AS solar_avg_inv_eff_pct,
    AVG(irradiance_wm2)      AS solar_avg_irradiance_wm2,
    AVG(panel_temperature_c) AS solar_avg_panel_temp_c
  FROM (
    SELECT
      plant_id,
      energy_mwh,
      power_mw,
      performance_ratio,
      inverter_efficiency_pct,
      irradiance_wm2,
      panel_temperature_c,
      window(timestamp, '1 hour') AS ts_win
    FROM solar_sensor_feed_silver
  )
  LATERAL VIEW explode(array(ts_win.start)) w AS window_start
  GROUP BY plant_id, window_start
),

-- 3) Wind KPIs per plant, per hour
wind_hourly AS (
  SELECT
    plant_id,
    window_start,
    SUM(energy_mwh)             AS wind_energy_mwh,
    AVG(power_mw)               AS wind_avg_power_mw,
    AVG(wind_speed_ms)          AS wind_avg_speed_ms,
    AVG(power_coefficient)      AS wind_avg_cp,
    AVG(tip_speed_ratio)        AS wind_avg_tsr,
    AVG(vibration_mms)          AS wind_avg_vibration_mms,
    SUM(CASE WHEN turbine_status = 'RUNNING' THEN 1 ELSE 0 END) /
    COUNT(*)::double            AS wind_availability_ratio
  FROM (
    SELECT
      plant_id,
      energy_mwh,
      power_mw,
      wind_speed_ms,
      power_coefficient,
      tip_speed_ratio,
      vibration_mms,
      turbine_status,
      window(timestamp, '1 hour') AS ts_win
    FROM wind_sensor_feed_silver
  )
  LATERAL VIEW explode(array(ts_win.start)) w AS window_start
  GROUP BY plant_id, window_start
)

SELECT
  COALESCE(b.plant_id, s.plant_id, w.plant_id)        AS plant_id,
  COALESCE(b.window_start, s.window_start, w.window_start) AS window_start,
  -- battery
  b.avg_soc_pct,
  b.min_soc_pct,
  b.max_soc_pct,
  b.avg_soh_pct,
  b.batt_energy_mwh,
  b.batt_avg_temp_c,
  b.batt_total_cycles,
  -- solar
  s.solar_energy_mwh,
  s.solar_avg_power_mw,
  s.solar_avg_pr,
  s.solar_avg_inv_eff_pct,
  s.solar_avg_irradiance_wm2,
  s.solar_avg_panel_temp_c,
  -- wind
  w.wind_energy_mwh,
  w.wind_avg_power_mw,
  w.wind_avg_speed_ms,
  w.wind_avg_cp,
  w.wind_avg_tsr,
  w.wind_avg_vibration_mms,
  w.wind_availability_ratio
FROM battery_hourly b
FULL OUTER JOIN solar_hourly s
  ON b.plant_id = s.plant_id AND b.window_start = s.window_start
FULL OUTER JOIN wind_hourly w
  ON COALESCE(b.plant_id, s.plant_id) = w.plant_id
 AND COALESCE(b.window_start, s.window_start) = w.window_start;
